## remove_uncertain_label.py
    - 불확실성이 높은 pseudo-label standard deviation에 따라 제거 후 self-training에 사용하기 

사용방법

    * git clone
    * cd 경로설정
    * 학습이 완료된 모델 path list로 생성
    * pseudo-label 해줄 데이터 pair를 list로 생성 
    * remove_uncertain_label.py 불러와서 실행

In [10]:
! git clone https://github.com/tommyEzreal/SolProject3-STS-Bechmark-improvement

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
'SolProject3-STS-Bechmark-improvement'에 복제합니다...
remote: Enumerating objects: 472, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 472 (delta 36), reused 0 (delta 0), pack-reused 405
오브젝트를 받는 중: 100% (472/472), 5.66 MiB | 4.30 MiB/s, 완료.
델타를 알아내는 중: 100% (283/283), 완료.


In [11]:
%cd /home/sol3sts/SolProject3-STS-Bechmark-improvement/label_selection/
%run remove_uncertain_label.py

/home/sol3sts/SolProject3-STS-Bechmark-improvement/label_selection
2022-12-30 18:04:17 - This is an info message


In [12]:
# pseudo-label해줄 unlabled data load 

unlabel = pd.read_csv('/home/sol3sts/dataset/aug_korsts.csv')
unlabel = unlabel[unlabel['source'] == 'korsts_test']
unlabel = unlabel[['sentence1','sentence2']]
unlabel

,sentence1,sentence2
2523,한 남자가 기타를 치고 있다.,한 남자가 트럼펫을 연주하고 있다.
2524,한 남자가 기타를 치고 있다.,한 남자가 트럼펫을 연주하고 있다.
2525,한 남자가 자전거를 타고 있다.,한 남자가 말하고 있다.
2526,한 남자가 양파를 자르고 있다.,한 여성이 호박을 썰고 있다.
2527,한 남자가 말하고 있다.,한 남자가 요리를 하고 있다.
...,...,...
3003,텍사스 공장 폭발로 100명 이상이 부상당했다.,수백 명의 사람들이 텍사스 비료 공장 폭발로 부상당한 것으로 추정된다.
3004,보도에 따르면 남한 군인은 북한과의 국경에서 5명의 동지들을 살해한 후 도망친다고 한다.,남한 군인이 동지들을 죽이고 도망친다.
3005,아프가니스탄의 무장 공격으로 상부에서 4명이 사망했다.,아프가니스탄의 무장 공격으로 발로치스탄에서 FC 병사가 사망했다.
3006,아프가니스탄 테러로 13명 사망,아프가니스탄 수도 모스크 공격으로 6명 사망


In [14]:
# n개의 모델과 라벨하고싶은 unlabeled data를 받는 list 각각 생성  


# unlabled data
sentence_pairs = list(zip(unlabel['sentence1'], unlabel['sentence2']))


# trained model list 
model_save_path_1 = '/home/sol3sts/cross_encoder_base_model_-2022-12-12_04-11-33'
model_save_path_2 = '/home/sol3sts/seed2_cross_encoder_-2022-12-12_06-14-43'
model_save_path_3 = '/home/sol3sts/seed3_cross_encoder_-2022-12-12_06-32-31'
model_save_path_4 = '/home/sol3sts/seed4_cross_encoder_-2022-12-12_06-49-54'
model_save_path_5 = '/home/sol3sts/seed5_cross_encoder_-2022-12-12_07-04-37'
model_save_path_6 = '/home/sol3sts/seed6_cross_encoder_-2022-12-13_03-54-27'
model_save_path_7 = '/home/sol3sts/seed7_cross_encoder_-2022-12-13_04-15-07'
model_save_path_8 = '/home/sol3sts/seed8_cross_encoder_-2022-12-13_04-28-38'
model_save_path_9 = '/home/sol3sts/seed9_cross_encoder_-2022-12-13_04-43-24'
model_save_path_10 = '/home/sol3sts/seed10_cross_encoder_-2022-12-13_04-58-40'




model_list = [model_save_path_1,
              model_save_path_2,
              model_save_path_3,
              model_save_path_4,
              model_save_path_5,
              model_save_path_6,
              model_save_path_7,
              model_save_path_8,
              model_save_path_9,
              model_save_path_10]

print("number of models used for ensemble:",len(model_list))
print("number of unlabeled data used for pseudo labeling:",len(sentence_pairs))

number of models used for ensemble: 10
number of unlabeled data used for pseudo labeling: 485


In [15]:
# remove_uncertain_label 함수사용해서 uncertainty가 높은 label을 제거한 labeled data 생성
ensembled_label = remove_uncertain_label(model_list,sentence_pairs)
len(ensembled_label)

2022-12-30 18:07:29 - Load model_0
None
2022-12-30 18:07:31 - Predict_with_model_0
None


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2022-12-30 18:07:33 - Load model_1
None
2022-12-30 18:07:34 - Predict_with_model_1
None


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2022-12-30 18:07:36 - Load model_2
None
2022-12-30 18:07:37 - Predict_with_model_2
None


...

2022-12-30 18:07:53 - Load model_8
None
2022-12-30 18:07:55 - Predict_with_model_8
None


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2022-12-30 18:07:56 - Load model_9
None
2022-12-30 18:07:58 - Predict_with_model_9
None


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

485

In [16]:
unlabel['ensembled_pseudo_label'] = ensembled_label
unlabel

,sentence1,sentence2,ensembled_pseudo_label
2523,한 남자가 기타를 치고 있다.,한 남자가 트럼펫을 연주하고 있다.,2.6
2524,한 남자가 기타를 치고 있다.,한 남자가 트럼펫을 연주하고 있다.,2.6
2525,한 남자가 자전거를 타고 있다.,한 남자가 말하고 있다.,1.9
2526,한 남자가 양파를 자르고 있다.,한 여성이 호박을 썰고 있다.,2.5
2527,한 남자가 말하고 있다.,한 남자가 요리를 하고 있다.,2.8
...,...,...,...
3003,텍사스 공장 폭발로 100명 이상이 부상당했다.,수백 명의 사람들이 텍사스 비료 공장 폭발로 부상당한 것으로 추정된다.,3.1
3004,보도에 따르면 남한 군인은 북한과의 국경에서 5명의 동지들을 살해한 후 도망친다고 한다.,남한 군인이 동지들을 죽이고 도망친다.,2.5
3005,아프가니스탄의 무장 공격으로 상부에서 4명이 사망했다.,아프가니스탄의 무장 공격으로 발로치스탄에서 FC 병사가 사망했다.,2.6
3006,아프가니스탄 테러로 13명 사망,아프가니스탄 수도 모스크 공격으로 6명 사망,3.0
